In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("lmqg/qg_annotation")

In [3]:
dataset = load_dataset("lmqg/qg_annotation")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 870.37it/s]


Dataset qg_annotation downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/lmqg___qg_annotation/qg_annotation/0.0.3/5fd9d4573e46b0d58528a31764696d51ac8a3415afeed228dd12eb3d4499a30e. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 464.64it/s]


In [6]:
dataset['test'][0]

{'correctness': 1.7999999523162842,
 'grammaticality': 3.0,
 'understandability': 2.4000000953674316,
 'prediction': 'What trade did the Ming dynasty have a shortage of?',
 'Bleu_4': 0.4961682856082916,
 'METEOR': 0.3572683334350586,
 'ROUGE_L': 0.7272727489471436,
 'BERTScore': 0.9142221808433533,
 'MoverScore': 0.6782580614089966,
 'reference_raw': 'What important trade did the Ming Dynasty have with Tibet?',
 'answer_raw': 'horse trade',
 'paragraph_raw': "Some scholars note that Tibetan leaders during the Ming frequently engaged in civil war and conducted their own foreign diplomacy with neighboring states such as Nepal. Some scholars underscore the commercial aspect of the Ming-Tibetan relationship, noting the Ming dynasty's shortage of horses for warfare and thus the importance of the horse trade with Tibet. Others argue that the significant religious nature of the relationship of the Ming court with Tibetan lamas is underrepresented in modern scholarship. In hopes of reviving th